# Histology Demo with Intel® Distribution of OpenVINO™ Toolkit. 

## Prerequisites
This sample requires the following:
- All files are present and in the following directory structure:
    
    - **histology.ipynb** - This Jupyter* Notebook
    - **data** - Directory to hold data
    - **python** -Directory for the Python scripts

## Introduction

This demo uses the [colorectal histology images dataset](https://www.tensorflow.org/datasets/catalog/colorectal_histology) to train a simple convolutional neural network in TensorFlow and demonstrates how to use OpenVINO™ integration with Tensorflow and OpenVINO™ Inference Engine to do inference on different Hardware architecture.

All images are RGB, 0.495 µm per pixel, digitized with an Aperio ScanScope (Aperio/Leica biosystems), magnification 20x. Histological samples are fully anonymized images of formalin-fixed paraffin-embedded human colorectal adenocarcinomas (primary tumors) from our pathology archive (Institute of Pathology, University Medical Center Mannheim, Heidelberg University, Mannheim, Germany).

The model is trained using the histology model introduced here: [Kather JN, Weis CA, Bianconi F, Melchers SM, Schad LR, Gaiser T, Marx A, Zollner F: Multi-class texture analysis in colorectal cancer histology (2016), Scientific Reports (in press)] (https://zenodo.org/record/53169#.X_T3iC1h10v)



### Key concepts
This sample application includes an example for the following:
- Application:
  - Load and visualize Tensorflow dataset
- Intel® DevCloud for the Edge: submitting jobs to perform on different edge compute nodes (rather than on the development node hosting this Jupyter* notebook)
  - Training jobs that train a convolutional neural network with Tensorflow V2
  - Running inference jobs with OpenVINO™ integration with Tensorflow
  - Running Inference jobs with OpenVINO™ Inference Engine
  - Monitoring job status
  - Viewing results and assessing performance for hardware on different compute nodes
- [Intel® Distribution of OpenVINO™ toolkit](https://software.intel.com/openvino-toolkit):
  - Create the necessary Intermediate Representation (IR) files for the inference model using [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html)
  - Run an inference application on multiple hardware devices using the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html)

### Install necessary packages

In [ ]:
!pip install -U pip==21.0.1
!pip install importlib_resources
!pip install openvino-dev==2021.4.2
!pip install tensorflow==2.7.0
!pip install openvino-tensorflow==1.1.0
!pip install -U tensorflow_datasets
!pip install ipywidgets

### Imports

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import tensorflow_datasets as tfds

print(tf.__version__)

In [ ]:
from platform import python_version
print(python_version())

 ### Dataset Examples
 #### How to load TF colorrectal_histology dataset

In [ ]:
os.environ['NO_GCE_CHECK'] = 'true'

In [ ]:
(ds), ds_info =  tfds.load('colorectal_histology', data_dir=".", 
                                          shuffle_files=True, split='train', 
                                          with_info=True, as_supervised=True)

assert isinstance(ds, tf.data.Dataset)
print(ds_info)

#### Display a few examples from the dataset

In [ ]:
x_key, y_key = ds_info.supervised_keys
ds_temp = ds.map(lambda x, y: {x_key: x, y_key: y})
tfds.show_examples(ds_temp, ds_info, plot_scale=5);
!mkdir test_data

#### Check the histology classification subtypes

In [ ]:
ds_info.features['label'].names

## Histology Demo
The Histology demo uses the Intel® Optimized Tensorflow to perform training and inference on input color rectal histology image dataset using OpenVINO™ integration with Tensorflow and Intel® Distribution of OpenVINO™ toolkit .  We will setup, train, run, and view the results for this application for several different hardware available on the compute nodes within the Intel® DevCloud for the Edge.  To accomplish this, we will be performing the following tasks:

1. Train the model in [Intel Optimized Tensorflow](https://software.intel.com/content/www/us/en/develop/tools/frameworks.html)
1. Run Inference using [OpenVINO™ Integration with Tensorflow](https://www.intel.com/content/www/us/en/developer/tools/devcloud/edge/build/ovtfoverview.html)
2. Use the [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html) to create the inference model IR files needed to perform inference
3. Create the job file used to submit running training and inference on compute nodes
4. View results and assess performance 

### How it works
At startup, for both training and inference, the Histology application configures itself by parsing the command line arguments. Once configured, the training application loads the specified input dataset, prepares the data accordingly and runs training on the specified edge compute node. Once the model is trained, the inference app loads the inference model's IR files into the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html) and runs inference on the specified input data to find the class of the given histology image. After the specific job is completed, the output is stored in the appropriate `results/<architecture>/` directory.  

The following sections will guide you through configuring and running the Histology demo.

### Model training for the histology images
The following sections will go through the steps to run training on the current EC2 instance. 

#### Create the job file

The job file is a [Bash](https://www.gnu.org/software/bash/) script that serves as a wrapper around the Python* executable of our application that can be executed directly on different types of edge compute node.  One purpose of the job file is to simplify running an application on different compute nodes by accepting a few arguments and then performing accordingly any necessary steps before and after running the application executable.  

For this sample, the job file we will be using is already written for you and appears in the next cell.  The job file will be submitted as if it were run from the command line using the following format:
```bash
training_job.sh <output_directory> <input_directory> <epochs> <device> <num_cores>
```
Where the job file input arguments are:
- <*output_directory*> - Output directory to use to store output files
- <*input_directory*> - Path to the input training data
- <*epochs*> - Epochs to train the model
- <*device*> - Specify the device used for training
- <*num_cores*> - If device is CPU, need to specify number of physical cores on the targeted device

Based on the input arguments, the job file will do the following:
- Change to the working directory `PBS_O_WORKDIR` where this Jupyter* Notebook and other files appear on the compute node
- Create the <*output_directory*>
- Run the application Python* executable with the appropriate command line arguments

Run the following cell to create the `training_job.sh` job file.  The [`%%writefile`](https://ipython.readthedocs.io/en/stable/interactive/magics.html#cellmagic-writefile) line at the top will write the cell contents to the specified job file `training_job.sh`.

In [ ]:
%%writefile training_job.sh

# Store input arguments: <output_directory> <input_directory>
INPUT_DIRECTORY=$1
OUTPUT_DIRECTORY=$2
EPOCHS=$3
DEVICE=$4
CORES=$5

# The default path for the job is the user's home directory,
#  change directory to where the files are.
if [ ! -d "./results/" ];then
   mkdir -p ./results
fi
# Make sure that the output directory exists.
mkdir -p $OUTPUT_FILE

# Install Tensorflow 
python python/run_training.py -i $INPUT_DIRECTORY \
                               -o $OUTPUT_DIRECTORY \
                               -e $EPOCHS \
                               -d $DEVICE \
                               -c $CORES


#### Submit training job

In [ ]:
! bash ./training_job.sh colorectal_histology results/xeon/cascade_lake 25 CPU 96

#### Trained model
Once the training job has completed, the queue system outputs the stdout and stderr streams, respecitvely.

The generated model file is written to the directory `results/<device>/checkpoints` that was specified as the output directory to the job file.  

<br><div class=danger><b>Wait!: </b>Please wait for the training job to complete before proceeding to the next step to do the OpenVINO™ model conversion and inferencing with OpenVINO™.</div>

### Inference using OpenVINO™ Integration with Tensorflow and OpenVINO™ Inference Engine
The following sections will go through the steps to run inference application for a single image using OpenVINO integration with Tensorflow and if dataset on OpenVINO Inference Engine. For viewing the results from job please refer to section 4.5

#### Inference job for single image using OpenVINO™ Integration with Tensorflow

##### Create the job file
Similarly to the training, we will run inference using OpenVINO™ integration with Tensorflow for single image.

The inference job file we will be using is already written for you and appears in the next cell.  The job file will be submitted as if it were run from the command line using the following format:
```bash
run_tensorflow_openvino.sh <device> 
```

Where the job file input arguments are:
- <*device*> - Specify the device used for inferencing

classification_ovtf_histology.py takes in multiple arguments as listed below
 - m "histology_model"
 - i "input_layer of the model>"
 - o "input_layer of the model>"
 - ip "image in Jpeg format"
 - l "label map file"
                  

Based on the input arguments, the job file will do the following:
- Create the <*output_directory*>
- Run the application Python* executable with the appropriate command line arguments



In [ ]:
%%writefile run_tensorflow_openvino.sh

#!/bin/sh

DEVICE=$1

python python/classification_ovtf_histology.py -m "./results/xeon/cascade_lake/frozen_histology.pb" \
                       -i "x" \
                       -o "Identity" \
                       -ip "./data/A8D0_CRC-Prim-HE-10_002c.tif_Row_1_Col_451.jpg" \
                       -it "image" \
                       -l "./data/labels_histology.txt" \
                       -f "openvino" \
                       --input_height 150 \
                       --input_width 150 \
                       --input_mean 3 \
                       -d $DEVICE 

##### Submit  Inference job using OpenVINO™ Integration with Tensorflow
 
The inference workload will run on the CPU. It can be modified to schedule on another accelerator by passing the flag F as with a value of GPU, VPU, or VAD-M.
 
The workload images are expected to be in JPEG format. Any custom inputs need to be converted to JPEG before performing inference on them.

In [ ]:
!chmod +x ./run_tensorflow_openvino.sh
! bash ./run_tensorflow_openvino.sh "CPU"

#### Convert the Tensorflow model to OpenVINO IR files
This workload is run on the CPU.

Run the following cell for Model Optimizer to create the model IR files. It converts the saved model trained from an edge Xeon Cascade Lake node to OpenVINO™ model format and saves it under `models/ov/<Model Precision>`. The model conversion can use either the generated savedModel or the frozen tensorflow model. Below mo_convert.sh script supports both model formats 

In [ ]:
input_model="saved_model" 

In [ ]:
!source mo_convert.sh $input_model

In [ ]:
input_model="frozen_model" 

In [ ]:
!source mo_convert.sh $input_model

In [ ]:
!echo "\nAll IR files that were created:"
!find ./models/ov/FP32 -name "*.xml" -o -name "*.bin"

#### Run inference on Histology Dataset via OpenVINO™ Inference Engine
The following sections will go through the steps to run our inference application for the Histology dataset. 

##### Create the job file
Similarly to the training, we will run inference on this EC2 instance node.

The inference job file we will be using is already written for you and appears in the next cell.  The job file will be submitted as if it were run from the command line using the following format:
```bash
inference_job.sh <model_directory> <input_directory> <output_directory> <device>
```
Where the job file input arguments are:
- <*model_directory*> - OpenVINO™ model directory
- <*input_directory*> - Path to the input testing data
- <*output_directory*> - Path to store the inference output
- <*device*> - Specify the device used for inferencing

Based on the input arguments, the job file will do the following:
- Create the <*output_directory*>
- Run the application Python* executable with the appropriate command line arguments

Run the following cell to create the `inference_job.sh` job file.  The [`%%writefile`](https://ipython.readthedocs.io/en/stable/interactive/magics.html#cellmagic-writefile) line at the top will write the cell contents to the specified job file `inference_job.sh`.

In [ ]:
%%writefile inference_job.sh

# Store input arguments: <model_type> <input_directory> <output_directory> <device>
MODEL_TYPE=$1
INPUT_DIRECTORY=$2
OUTPUT_DIRECTORY=$3
DEVICE=$4

# Make sure that the output directory exists.
mkdir -p $OUTPUT_DIRECTORY

# Run the inference code
python python/run_inference.py -d $DEVICE \
                                -i $INPUT_DIRECTORY \
                                -o $OUTPUT_DIRECTORY \
                                -m $MODEL_TYPE

##### Run inference
In the cell below, we run the inference workload .

In [ ]:
! chmod +x ./inference_job.sh
! bash ./inference_job.sh FP32 test_data/testdata.npz results/xeon/cascade_lake/ CPU

In [ ]:
!find "results/xeon/cascade_lake/" -name "stats.txt"

### View inferencing results

The output classification results for each job is written to the file `stats_<*job id*>.txt` located in the directory `results/<device>` that was specified as the output directory to the job file. Each line of the result txt file records the following information:

`index of the testing image, true class label, predicted class label`

Example of how to view the inferencing results are shown below. 


In [ ]:
#For viewing results
output_file_clx = "results/xeon/cascade_lake/"+"stats"+".txt"
fd = open( output_file_clx, 'r')
print(fd.read())

## Quantization

Quantization is the process of reducing the model's precision. By performing this optimization, you can accelerate your model execution time.  

The [quantize.py](python/quantize.py) file contains quantization script and uses the [post training optimization toolkit (POT) API](https://docs.openvinotoolkit.org/latest/pot_compression_api_README.html) to reduce model's precision from FP32 to INT8. Quantization settings, such as the path to original model, path to dataset, quantization algorithm etc., which should be set via configs. ```DatasetsDataLoader``` creates quantization dataset from the sample video and loads one by one input images to POT, when quantization process starts. When quantization is finished, the INT8 model will be saved at ```'/models/ov/INT8'``` directory.

Run the following cell to create the ```quantization_job.sh``` job file. This script runs quantization and benchmarking of the quantized and non-quantized models to compare their execution time.

In [ ]:
%%writefile quantization_job.sh

# Run the quantization script
python3.6 ./python/quantize.py

# Run the benchmark_app for FP32 model
benchmark_app \
        -m ./models/ov/FP32/saved_model.xml 2>/dev/null | grep Throughput | xargs echo FP32

# Run the benchmark_app for INT8 model
benchmark_app \
        -m ./models/ov/INT8/saved_model.xml 2>/dev/null | grep Throughput | xargs echo INT8

Run the following cell to start quantization and benchmarking of the quantized model.

In [ ]:
import os
import time

In [ ]:
def wait_for_job_to_finish(job_id):
    if job_id:
        print("Job submitted to the queue. Waiting for it to complete.")    
                    
        # Reading the benchmark_app results
        print('Results for FP32 and INT8 models:')
        for line in job_id:            
            if 'Throughput:' in line:
                print(line.split('\n')[0])
                      
    else:
        print("Error in job submission.")

job_id_core = get_ipython().getoutput('bash ./quantization_job.sh')
benchmarks = wait_for_job_to_finish(job_id_core)